<a href="https://colab.research.google.com/github/nickradunovic/ragdemo_politie/blob/main/rechtspraak_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval-Augmented Generation (RAG) with Public Justice Data

# How RAG Works
Traditional LLMs like GPT-4 come pre-trained on massive public datasets, allowing for incredible natural language processing capabilities out of the box. However, their utility is limited without access to your own private data. RAG tries to combat this limitation by introducing a retrieval mechanism that allows the model to access externally provided data sources like private data. Here's a simplified overview of how it works:

- **Retrieval**: The model performs an initial retrieval step to gather contextually relevant information from a knowledge base or external documents.

- **Augmentation**: The retrieved information is then seamlessly integrated into the generation process, augmenting the model's understanding and improving the quality of generated content.

- **Generation**: The model generates text, now equipped with the additional knowledge obtained through retrieval, resulting in more informed and contextually rich output.

In this demo, we will be working with [public case law data](https://uitspraken.rechtspraak.nl/resultaat?zoekterm=&inhoudsindicatie=&publicatiestatus=ps1&sort=Relevance&rechtsgebied=r3) containing records of lawsuit rulings of criminal law. We will demonstrate how RAG can be used to retrieve relevant records and generate accurate, context-aware responses to queries about this data. This will showcase the behind-the-scenes workings of RAG and illustrate how it can add significant value over standard LLM implementations that do not utilize a retrieval mechanism.

## Objectives
By the end of this tutorial, you will:

- Understand the core concepts of RAG and its advantages over standard LLMs.

- Learn how to implement a RAG system using public justice data.

- See practical examples of querying the data and retrieving contextually relevant responses.

# Create the LLM prompt with question and answer

Put here your OAI keys

In [1]:
# copy/paste keys here

Install required dependencies

In [2]:
%%writefile requirements.txt
python-dotenv
openai
azure-search-documents
azure-core

Overwriting requirements.txt


In [3]:
!pip install -r requirements.txt

Import Required Dependencies

In [4]:
import uuid
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import List
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

### Initiating the Azure SearchClient and AzureOpenAI instance

In this first step, we will set up our environment and connect to the necessary Azure services. This involves loading environment variables and initializing clients for Azure Cognitive Search and Azure OpenAI. These connections are crucial as they allow us to interact with the search index and the OpenAI models.

In [5]:
load_dotenv()

credential = AzureKeyCredential(os.environ.get("SEARCH_KEY"))
search_client = SearchClient(
    endpoint=os.environ.get("SEARCH_ENDPOINT"),
    index_name=os.environ.get("SEARCH_INDEX_NAME"),
    credential=credential,
)
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_ENDPOINT"),
)

### Implementing the Assistant Class

In this step, we will implement the Assistant class, which serves as the core of our RAG system. This class will manage interactions with the Azure Cognitive Search and Azure OpenAI services. It includes methods for searching documents and generating responses to user queries.

In [64]:
class Assistant:
    def __init__(self, search_client: SearchClient, openai_client):
        self.search_client = search_client
        self.llm = openai_client.chat.completions.create
        self.embedding = openai_client.embeddings.create

    def search(self, query: str, top_n_documents: int) -> List:
        """
        Searches for documents based on the given query.

        Args:
            query (str): The search query.
            top_n_documents (int): Number of top documents to retrieve.

        Returns:
            list: A list of search results.
        """
        query_embedding = (
            self.embedding(input=[query], model=os.getenv("EMBEDDING_NAME"))
            .data[0]
            .embedding
        )

        # Azure AI search requires a vector query
        vector_query = VectorizedQuery(
            vector=query_embedding,
            k_nearest_neighbors=top_n_documents,
            fields="content_vector",
            exhaustive=True,
        )

        # passing in query in search_text makes it 'hybrid' search
        search_results = self.search_client.search(
            search_text=query, vector_queries=[vector_query], top=top_n_documents
        )

        return search_results

    def rag_chat(self, question: str) -> str:
        """
        Ask a question to the RAG chatbot

        Args:
            question (str): The question to ask the chatbot.

        Returns:
            str: The response from the chatbot.
        """
        search_results = self.search(question, top_n_documents=7)

        documents_string = ""
        for result in search_results:
            documents_string += (
                f"ECLI: {result['title']} | Text: {result['content']}\n\n"
            )

        response = self.llm(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": f"Je bent een assistent die vragen beantwoordt over Rechtspraken. \
                        Bij elke vraag krijg je relevante info van verschillende rechtszaken meegestuurd in CONTEXT. \
                        De vraag staat bij VRAAG. \
                        Je moet je antwoord enkel en alleen baseren op de meegestuurde info \
                        Refereer in je antwoord per stuk. \
                        VRAAG: {question}. CONTEXT:\n{documents_string}. \nJOUW ANTWOORD:",
                },
            ],
        )

        return response.choices[0].message.content, documents_string

    def normal_chat(self, question: str) -> str:
        """
        Ask a question to the RAG chatbot

        Args:
            question (str): The question to ask the chatbot.

        Returns:
            str: The response from the chatbot.
        """
        response = self.llm(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "Je bent een assistent die vragen beantwoordt over Rechtspraken.",
                },
                {"role": "user", "content": f"VRAAG: {question}"},
            ],
        )

        return response.choices[0].message.content

### Instantiating the Assistant Class
Now that we have defined the Assistant class, the next step is to create an instance of this class. This will allow us to use the methods we defined to perform searches and generate responses. By passing in the search_client and client (initialized in the previous step) to the Assistant constructor, we can set up our assistant for use.

In [65]:
assistant = Assistant(search_client, client)

### Generating a Response Using the RAG System
Now, we will utilize the Assistant class instance to generate a response to a specific questions related and not related to legal cases. See whether or not there is a difference in response quality between quantitative and substantive/content question. Try out your own questions as well.

In [67]:
vraag = "welke rechtszaken hebben een man als dader?"
rag_antwoord, rag_documents = assistant.rag_chat(vraag)
print(rag_antwoord)

De volgende rechtszaken hebben een man als dader:

1. ECLI: NL:RBDHA:2021:11578
   - Verdachte: Een man geboren op [geboortedatum], zaaknummer 09/240249-20.
   - Feit: Verkrachting.

2. ECLI: NL:GHARL:2021:11939
   - Verdachte: Een man geboren te [geboorteplaats] op [geboortedatum] 1993.
   - Feit: Poging tot ontploffing en diefstal door braak.

3. ECLI: NL:RBDHA:2021:219
   - Verdachte: Een man geboren op [geboortedag] 1961.
   - Feit: Poging tot moord.

4. ECLI: NL:RBZWB:2023:9196
   - Verdachte: Een man geboren op [geboortedag] 2003.
   - Feit: Ontvoering en diefstal met geweld.

5. ECLI: NL:GHARL:2023:2096
   - Verdachte: Een man geboren te [geboorteplaats] op [geboortedag] 1995.
   - Feit: Poging tot doodslag en bedreiging.

6. ECLI: NL:RBDHA:2023:17261
   - Verdachte: Een man geboren op [geboortedag] 1985.
   - Feit: Zware mishandeling met voorbedachte raad.

7. ECLI: NL:RBNNE:2022:4871
   - Verdachte: Een man geboren op [geboortedatum] 1990.
   - Feit: Mishandeling, bedreiging, 

In [68]:
vraag = "in welke rechtszaken komt het gebruik van een vuurwapen voor? Het vuurwapen hoeft niet te zijn afgegaan."
rag_antwoord, rag_documents = assistant.rag_chat(vraag)
print(rag_antwoord)

Hieronder staan de rechtszaken waarin het gebruik van een vuurwapen voorkomt:

1. **ECLI:NL:RBROT:2020:10973**
   - In deze zaak ging het om het voorhanden hebben van een vuurwapen, een pistool van het merk FN type 1922, kaliber 7,65 millimeter. Er werd een pistool gevonden achter het dashboardkastje in een auto, gevuld met munitie (CONTEXT).

2. **ECLI:NL:GHDHA:2020:2493**
   - Hier betreft het een schietincident op 26 december 2016 te Hellevoetsluis, waarbij met vuurwapens een of meer personen van het leven werden beroofd. Verschillende kaliber hulzen zijn gevonden en getuigen verklaarden over het afvuren van kogels (CONTEXT).

3. **ECLI:NL:RBOVE:2022:2701**
   - In deze zaak betrof het een verdenking van het voorhanden hebben van een automatisch vuurwapen (machinepistool van het merk IMI, type Micro UZI, kaliber 9 x 19 mm) en een pistool van het merk Taurus, type PT92, kaliber 9 mm kort (.380) (CONTEXT).

4. **ECLI:NL:RBZWB:2020:3862**
   - De feiten omvatten afpersing waarbij het g

In [74]:
vraag = "In welke rechtzaken zijn kinderen betrokken?"
rag_antwoord, rag_documents = assistant.rag_chat(vraag)
print(rag_antwoord)

Op basis van de meegestuurde context zijn er kinderen betrokken in de volgende rechtszaken:

1. **Rechtbank Den Haag (ECLI:NL:RBDHA:2021:467)**:
   - Minderjarige 1, geboren op [geboortedatum 1] 2010 te [geboorteplaats 1], Groot-Brittannië.
   - Minderjarige 2, geboren op [geboortedatum 2] 2011 te [geboorteplaats 2], Groot-Brittannië.

2. **Gerechtshof Amsterdam (ECLI:NL:GHAMS:2024:26)**:
   - [Minderjarige 1] Olij.

3. **Rechtbank Rotterdam (ECLI:NL:RBROT:2020:9518)**:
   - [Naam minderjarige 1], geboren op [geboortedatum minderjarige 1] 2005 te [geboorteplaats].
   - [Naam minderjarige 2], geboren op [geboortedatum minderjarige 2] 2006 te [geboorteplaats].

4. **Rechtbank Den Haag (ECLI:NL:RBDHA:2024:6652)**:
   - [Kind 1], geboren op [geboortedag 1] 2015 te [geboorteplaats 1].
   - [Kind 2], geboren op [geboortedag 2] 2020 te [geboorteplaats 2].

5. **Gerechtshof Arnhem-Leeuwarden (ECLI:NL:GHARL:2023:623)**:
   - [De minderjarige 1], geboren [in] 2013 te [woonplaats 2].
   - [De min

In [77]:
vraag = "Bij welke case waren de meeste doden?"
# vraag = "Hoeveel doden waren er bij de case met de meeste doden?"
rag_antwoord, rag_documents = assistant.rag_chat(vraag)
print(rag_antwoord)

Bij de case met de meeste doden volgens de aangeleverde rechtszaken was er sprake van drie doden. Dit vindt plaats in de zaak met ECLI: ECLI:NL:RBROT:2020:904 (Rechtbank Rotterdam).

- De rechtbank stelt dat er onvoldoende bewijs is om de verdachten te veroordelen voor de moord op [naam slachtoffer]. Er zijn indicaties en aanwijzingen maar de concrete bewijsstukken ontbreken, vandaar is de verdachte niet schuldig geacht aan moord en het wegmaken van het stoffelijk overschot.


### How to handle unrelated questions?

At the moment, the RAG Assistant allows users to ask questions unrelated to the data in the vector database. Imagine a use case where this behaviour is not acceptable. How can we ensure that the RAG assistent gives appropiate responses and answers questions only related to the use case?

In [87]:
vraag = "Hoe kan het zijn dat appels verschillende kleuren hebben (bijvoorbeeld rood, groen, geel of een combinatie van deze kleuren)?"
rag_antwoord, rag_documents = assistant.rag_chat(vraag)
print(rag_antwoord)

Het kan voorkomen dat appels verschillende kleuren hebben vanwege een combinatie van diverse factoren, waaronder genetica, milieuomstandigheden en de rijpingstijd.

1. **Genetica**: Verschillende appelrassen hebben genetische verschillen die zorgen voor variatie in kleur. Genetische eigenschappen bepalen bijvoorbeeld of een appel rood, groen of geel is, of een combinatie van deze kleuren.

2. **Milieuomstandigheden**: De omgeving waarin de appels groeien, zoals het klimaat en de hoeveelheid zonlicht die de appels ontvangen, kan invloed hebben op de uiteindelijke kleur. Zonlicht kan bijvoorbeeld zorgen voor meer roodheid in appels doordat het de productie van anthocyanen stimuleert, een soort pigment dat voor rode, paarse, en blauwe kleuren zorgt.

3. **Rijpingstijd**: De mate van rijpheid kan ook bijdragen aan de kleurvariatie in appels. Naarmate appels rijpen, kunnen ze van kleur veranderen.

In termen van rechtbankverslagen en juridische context werden geen specifieke gevallen gevond

**TASK**: Reprogram the RAG Assistant so that it does not answer questions unrelated to legal cases.

Does it work? Discuss why it does / why it doesn't.

In [ ]:
# Write a query unrelated to legal case law data to test whether or not the RAG Assistent answers this kind of questions.

### Utilize the 'normal chat' that doesn't incorporate RAG functionality

If you would like to ask questions to the OAI model without RAG functionality, make an OAI call using the `normal_chat` method of the Assistant. Using this non-RAG chat one can play around and try to spot differences between a LLM with RAG functionality and one without. 

**TASK**: What are the use cases and down- and upsides of using RAG compared to using a regular LLM?

In [ ]:
# Instead of using `rag_chat`, write queries using the `normal_chat` method of the Assistant instead. Try out various queries and compare with the `rag_chat` method.

### Chunk size

Chunk size is a parameter that determines the length of each segment (or "chunk") of text that the source documents are divided into. In our implementation, the chunk size is specified in terms of the number of characters. The embeddings in the vector store we just used where created using a chunk size of `20480`, which is large enough for each legal case. However, what would happen if we use a smaller chunk size like `1024`? Now we use the search_index_name `SEARCH_INDEX_NAME=index-politiedemo` that was made with a large (`20480`) chunk size.

We prepared some indexes in our vector store for you to play with. Change the search_index_name to `SEARCH_INDEX_NAME=index-politiedemo-s` so we can see the effect of having a smaller chunk size of `1024`.

In [59]:
vraag = ""  # enter your question
rag_antwoord, rag_documents = assistant.rag_chat(vraag)
print(rag_antwoord)

{'content': '\n\nRECHTBANK ZEELAND-WEST-BRABANT\n\n\nStrafrecht\n\n\n\nZittingsplaats: Breda\n\n\n\nparketnummer: 02/128812-21  \n\n\n\n\nvonnis van de meervoudige kamer van 4 april 2022\n\n\n\n\nin de strafzaak tegen \n\n\n\n\n\n          [verdachte]\n        \n\ngeboren op [geboortedag] 1995 te [plaats] \nwonende te [adres]\nraadsvrouw mr. F.L. Donders, advocaat te Tilburg\n\n\n\n\n\n1Onderzoek van de zaak\nDe zaak is inhoudelijk behandeld op de zitting van 21 maart 2022, waarbij de officier van justitie, mr. A.I.M.M. Gudde, en de verdediging hun standpunten kenbaar hebben gemaakt. Namens de bepaalde partij [slachtoffer] heeft mr. N.M.E. Verpaalen, advocaat te Breda, de vordering tot schadevergoeding toegelicht.\n\n\n\n\n2De tenlastelegging\n\nDe tenlastelegging is als bijlage I aan dit vonnis gehecht. \nDe verdenking komt er, kort en feitelijk weergegeven, op neer dat verdachte samen met anderen heeft geprobeerd [slachtoffer] te doden door hem te slaan en te schoppen, dan wel dat hi

### Changing the `top_n_documents`

By using a smaller chunk size, it might be possible to retrieve a larger number of documents from the vector database. At the moment, `top_n_documents = 7`. Try retrieving a larger number of documents per query as see how this impacts the response quality. 

**NOTE**: make sure to use the index created using the smaller chunk size of `1024`.

In [ ]:
# write a query for the RAG Assistant while retrieving more than 7 documents.

### Changing the OAI model

There are different LLMs one can use for their RAG-implementation as well. We just used OpenAI GPT-4o in the RAG-assistant we just played with. However, you can imagine use cases and projects that opt to using a model not from OpenAI. For this demo, we keep on using OpenAI but we can play with different models they provide.

Next to `gpt-4o`, we also deployed `gpt-35-turbo` and `gpt-4` for this demo. You can change the model to one of these other OAI models within the RAG assistant by changing its code. Note, that you can change the model for either the `rag_chat` and the `normal_chat` methods. What would be reasons to opt for one LLM over the other?

In [ ]:
# Change the OAI model used in the RAG-assistant, and try out the RAG-assistant to see if you can find any differences

## Things to consider and try out for yourself:
- Chunk size: why is it important, what does it do. Discuss that we return best 7 chunks and how this relates to chunk size
- OAI model chatGPT-4o vs other OAI model
- Language barrier of RAG application: we can use Dutch language to query documents written in English or other language
- How can we utilize system prompt to get the response that we want in our application (can we include any guard rails?)
- What could be ways to enforce that the RAG Assistant is consistant in its responses?
- Difference in quality of response between quantitative questions (how many ...?, what is the greatest ...?) and qualitative (inhoudelijke) questions. 
- What could be shortcomings of this RAG implementation? --> (1) no chat history is saved at the moment, (2) ..., ...

In [ ]:
# Try out your own queries after playing with one of the points mentioned above.